# Introduction to Classification

Classification involves using labeled (known) training examples to predict a class label for unseen input examples. In this lab we will use the classification functionality provided by the *scitkit-learn* Python package.

In [3]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler

## Data Loading and Preparation

For the examples in this notebook, we will use the *Penguin* dataset described [here](https://www.kaggle.com/parulpandey/penguin-dataset-the-new-iris).

The dataset contains records related to penguins collected from islands in the Palmer Archipelago, Antarctica. Each penguin belongs to one of 3 species (Adelie', 'Gentoo', 'Chinstrap'). We would like to train a classification algorithm to automatically classify a record describing a penguin into one of the three species (classes).

The features in the data are as follows:

- *island*: The name of the island in Antarctica where the penguin was found ('Dream', 'Torgersen', or 'Biscoe') 
- *bill_length*: Length of the penguin's bill in mm
- *bill_depth*: Depth of the penguin's bill in mm
- *flipper_length*: Length of the penguin's flipper in mm
- *body_mass*: Weight of the penguin in grams
- *sex*: Indicates whether the penguin is male or female
- *species*: The species of this penguin (either 'Adelie', 'Gentoo', or 'Chinstrap')


In [6]:
df = pd.read_csv('penguins.csv')
df.head()

,island,bill_length,bill_depth,flipper_length,body_mass,sex,species
0,Dream,37.0,16.5,185,3400,female,Adelie
1,Biscoe,44.4,17.3,219,5250,male,Gentoo
2,Biscoe,49.0,16.1,216,5550,male,Gentoo
3,Biscoe,50.0,16.3,230,5700,male,Gentoo
4,Torgersen,40.2,17.0,176,3450,female,Adelie


Each example in the dataset has a class label or a "target" from three possible classes. We can look at the distribution of these classes (i.e. the number of penguins in each species):

In [8]:
df["species"].value_counts()

species
Adelie       146
Gentoo       119
Chinstrap     68
Name: count, dtype: int64

For the purposes of classification, we will focus on the numeric features in the data and remove the non-numeric features ('island' and 'sex'). We will also separate out the feature values from the target label ('species').

In [10]:
target = df["species"].values
data = df[["bill_length", "bill_depth", "flipper_length", "body_mass"]]
data.head()

,bill_length,bill_depth,flipper_length,body_mass
0,37.0,16.5,185,3400
1,44.4,17.3,219,5250
2,49.0,16.1,216,5550
3,50.0,16.3,230,5700
4,40.2,17.0,176,3450


Since the various numeric features in the data all have different ranges, we will apply min-max normalisation to transfrom them in to the range [0,1]. We can use the *MinMaxScaler* in scikit-learn to do this. Note the output will be a NumPy array.

In [12]:
normalizer = MinMaxScaler()  
data_scaled = normalizer.fit_transform(data.values)
data_scaled

array([[0.17818182, 0.4047619 , 0.22033898, 0.19444444],
       [0.44727273, 0.5       , 0.79661017, 0.70833333],
       [0.61454545, 0.35714286, 0.74576271, 0.79166667],
       ...,
       [0.75272727, 0.82142857, 0.55932203, 0.51388889],
       [0.25090909, 0.66666667, 0.22033898, 0.26388889],
       [0.69818182, 0.80952381, 0.44067797, 0.27777778]])

## Basic Classification

As our classification algorithm in this notebook, we will focus on the use of the simple but effectives *k-Nearest Neighbour (KNN) classifier*. Given a new input example, this algorithm finds the most similar previous examples for which a decision has already been made (i.e. their nearest neighbours from the training set). Based on the majority vote among the K neighbours, a prediction is made for the input.

Build a nearest neighbour classifier using just 1 nearest neighbour. 
In this case we will use the full dataset and all of the target labels for those examples.  
Because the ranges of the feature values are quite different we scale them all to the range [0,1] so they all have the same impact on classification.

In [34]:
knn = KNeighborsClassifier(n_neighbors=1)
m = knn.fit(data_scaled, target)

We can test it out by making a prediction for a new input example described by 4 feature values:

In [36]:
# create our new penguin record
penguin1 = [39.1, 16.8, 180.5, 3705.0]
unseen_data = np.array([penguin1])
# apply the same min-max- normalisation as before
unseen_scaled = normalizer.transform(unseen_data)

In [38]:
# make the prediction, the output is the class label
prediction = knn.predict(unseen_scaled)
prediction[0]

'Adelie'

We can also make predictions for multiple unseen input examples at once:

In [40]:
penguin1 = [39.1, 16.8, 180.5, 3705]
penguin2 = [46.2, 14.9, 208, 5286]
penguin3 = [50.3, 18.8, 201.5, 3804]
penguin4 = [40.1, 17.3, 185, 3402]
unseen_data = np.array([penguin1, penguin2, penguin3, penguin4])
# normalize the input data
unseen_scaled = normalizer.transform(unseen_data)
# make the predictions for the 4 unseen examples
knn.predict(unseen_scaled)

array(['Adelie', 'Gentoo', 'Chinstrap', 'Adelie'], dtype=object)

## Training and Test Sets

A key task when applying a classifier is to determine how effective our classifier will be at making predictions. One way to estimate this is to divide the full dataset into two sets using a "hold-out strategy":
1. *Training set*: A set of examples used to build the classification model.
2. *Test set*: A separate set of examples that is withheld from the classifier during training, and is used afterwards to evaluate the model.

To evaluate the effectiveness of our KNN classifier on the penguin data, we will randomly split the complete dataset into a training test (used to build the model) and an unseen test set (used to try out and evaluate the model). Scikit-learn provides functionality to do this. We will specify that 30% (0.3) of the data will be used for the test set. By specifiying a value for the *random_state*, we can reproduce the same results again.

In [42]:
from sklearn.model_selection import train_test_split
# use 70% for training, 30% for testing
data_train, data_test, target_train, target_test = train_test_split(data_scaled, target, 
    test_size=0.3, random_state=1)
print("Training set has %d examples" % data_train.shape[0])
print("Test set has %d examples" % data_test.shape[0])

Training set has 233 examples
Test set has 100 examples


Once we have performed our split, we then train our model based only on the training data:

In [44]:
model = KNeighborsClassifier(n_neighbors=1)
m = model.fit(data_train, target_train)

Make predictions for the test set, based on the model that we just trained:

In [46]:
predicted = model.predict(data_test)
# print the predictions
print("Predictions:\n%s" % predicted)
# print the number of predictions from each class
print("Class counts:\n%s" % pd.Series(predicted).value_counts())

Predictions:
['Chinstrap' 'Chinstrap' 'Adelie' 'Gentoo' 'Adelie' 'Adelie' 'Adelie'
 'Adelie' 'Adelie' 'Adelie' 'Gentoo' 'Chinstrap' 'Adelie' 'Gentoo'
 'Gentoo' 'Adelie' 'Chinstrap' 'Gentoo' 'Chinstrap' 'Chinstrap' 'Adelie'
 'Chinstrap' 'Adelie' 'Chinstrap' 'Gentoo' 'Adelie' 'Gentoo' 'Gentoo'
 'Gentoo' 'Gentoo' 'Adelie' 'Adelie' 'Gentoo' 'Gentoo' 'Chinstrap'
 'Chinstrap' 'Adelie' 'Chinstrap' 'Adelie' 'Gentoo' 'Gentoo' 'Chinstrap'
 'Adelie' 'Adelie' 'Chinstrap' 'Chinstrap' 'Chinstrap' 'Gentoo' 'Adelie'
 'Chinstrap' 'Adelie' 'Adelie' 'Gentoo' 'Chinstrap' 'Chinstrap' 'Adelie'
 'Gentoo' 'Gentoo' 'Adelie' 'Chinstrap' 'Gentoo' 'Gentoo' 'Adelie'
 'Gentoo' 'Gentoo' 'Gentoo' 'Adelie' 'Adelie' 'Adelie' 'Gentoo' 'Adelie'
 'Adelie' 'Adelie' 'Adelie' 'Chinstrap' 'Adelie' 'Adelie' 'Adelie'
 'Adelie' 'Adelie' 'Gentoo' 'Gentoo' 'Adelie' 'Gentoo' 'Adelie' 'Gentoo'
 'Gentoo' 'Adelie' 'Chinstrap' 'Adelie' 'Adelie' 'Gentoo' 'Gentoo'
 'Adelie' 'Gentoo' 'Adelie' 'Gentoo' 'Adelie' 'Chinstrap' 'Chinstrap']
Cla

We can compare our predictions to the "correct answer" based on the labels for the test data:

In [48]:
print("Predictions\n", predicted)
print("Correct labels\n", target_test)

Predictions
 ['Chinstrap' 'Chinstrap' 'Adelie' 'Gentoo' 'Adelie' 'Adelie' 'Adelie'
 'Adelie' 'Adelie' 'Adelie' 'Gentoo' 'Chinstrap' 'Adelie' 'Gentoo'
 'Gentoo' 'Adelie' 'Chinstrap' 'Gentoo' 'Chinstrap' 'Chinstrap' 'Adelie'
 'Chinstrap' 'Adelie' 'Chinstrap' 'Gentoo' 'Adelie' 'Gentoo' 'Gentoo'
 'Gentoo' 'Gentoo' 'Adelie' 'Adelie' 'Gentoo' 'Gentoo' 'Chinstrap'
 'Chinstrap' 'Adelie' 'Chinstrap' 'Adelie' 'Gentoo' 'Gentoo' 'Chinstrap'
 'Adelie' 'Adelie' 'Chinstrap' 'Chinstrap' 'Chinstrap' 'Gentoo' 'Adelie'
 'Chinstrap' 'Adelie' 'Adelie' 'Gentoo' 'Chinstrap' 'Chinstrap' 'Adelie'
 'Gentoo' 'Gentoo' 'Adelie' 'Chinstrap' 'Gentoo' 'Gentoo' 'Adelie'
 'Gentoo' 'Gentoo' 'Gentoo' 'Adelie' 'Adelie' 'Adelie' 'Gentoo' 'Adelie'
 'Adelie' 'Adelie' 'Adelie' 'Chinstrap' 'Adelie' 'Adelie' 'Adelie'
 'Adelie' 'Adelie' 'Gentoo' 'Gentoo' 'Adelie' 'Gentoo' 'Adelie' 'Gentoo'
 'Gentoo' 'Adelie' 'Chinstrap' 'Adelie' 'Adelie' 'Gentoo' 'Gentoo'
 'Adelie' 'Gentoo' 'Adelie' 'Gentoo' 'Adelie' 'Chinstrap' 'Chinstrap']
Cor

We can quantitatively check how accurate these predictions are, by measuring *accuracy*, which will return a value between 0.0 (predictions are completely wrong) and 1.0 (predictions are 100% accurate):

In [50]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(target_test, predicted)
print("Accuracy=%.3f" % acc)

Accuracy=0.980


## Selecting Parameters

Many classification algorithms have one or more parameter values which control various aspects of the learning process. In the case of a KNN classifier, the key parameter is the number of neighbours *k* considered when making a prediction. Different values for this parameter can lead to different predictions on unseen data, resulting in higher levels of accuracy.

Using the training/test split that we created above, we will examine the effect of increasing the parameter for the number of neighbours *k* on the accuracy of our predictions.

In [52]:
# iterate over a range of values of k
for k in range(1, 16):
    # train a classifier with this parameter value
    model = KNeighborsClassifier(n_neighbors=k)
    m = model.fit(data_train, target_train)
    # make predictions
    predicted = model.predict(data_test)
    # evaluate the predictions
    acc = accuracy_score(target_test, predicted)
    print("K=%02d neighbours: Accuracy=%.3f" % (k, acc))

K=01 neighbours: Accuracy=0.980
K=02 neighbours: Accuracy=1.000
K=03 neighbours: Accuracy=1.000
K=04 neighbours: Accuracy=1.000
K=05 neighbours: Accuracy=1.000
K=06 neighbours: Accuracy=0.990
K=07 neighbours: Accuracy=0.990
K=08 neighbours: Accuracy=0.990
K=09 neighbours: Accuracy=0.990
K=10 neighbours: Accuracy=0.990
K=11 neighbours: Accuracy=1.000
K=12 neighbours: Accuracy=0.990
K=13 neighbours: Accuracy=0.980
K=14 neighbours: Accuracy=0.980
K=15 neighbours: Accuracy=0.980


We see a little variation in the accuracy results above, although for this dataset the algorithm is not very sensitive to the choice of parameter value for KNN. However, this may not be the case for more difficult classification tasks and more complex datasets.